In [59]:
import pymysql
import sqlalchemy
import mysql.connector
from getpass import getpass
import pandas as pd

import getpass

In [60]:
password = getpass.getpass("Introduce tu contraseña para la base de datos MySQL: ")

Introduce tu contraseña para la base de datos MySQL:  ········


In [61]:
#1. Establish a connection between Python and the Sakila database.
conn = mysql.connector.connect(host = "localhost",
                               user = "root",
                               password = password,
                              database = 'sakila')

In [62]:
conn.is_connected()

True

In [63]:
#2. 2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year 
    #(passed as parameters) from the Sakila database as a Pandas DataFrame. 
    #The function should take in three parameters:

    #- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
    #- `month`: an integer representing the month for which rental data is to be retrieved.
    #- `year`: an integer representing the year for which rental data is to be retrieved.


In [64]:
import pandas as pd
import mysql.connector

def rentals_month(conn, month, year):
    """
    Recupera datos de alquiler para un mes y año dados de la base de datos Sakila,
    utilizando una conexión mysql.connector.

    Parámetros:
    - conn: Conexión a la base de datos Sakila a través de mysql.connector.
    - month: Entero que representa el mes para el cual se desea recuperar los datos de alquiler.
    - year: Entero que representa el año para el cual se desea recuperar los datos de alquiler.

    Retorna:
    - Un DataFrame de pandas con los datos de alquiler del mes y año especificados.
    """

    query = f"""
    SELECT 
        rental_id, customer_id, staff_id, rental_date, return_date, inventory_id
    FROM 
        rental
    WHERE 
        MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    
    # Usar el cursor para ejecutar la consulta
    cursor = conn.cursor()
    cursor.execute(query)
    
    # Extraer datos y convertir a DataFrame
    column_names = [i[0] for i in cursor.description]
    rental_records = cursor.fetchall()
    df_rentals = pd.DataFrame(rental_records, columns=column_names)
    
    # Cerrar cursor
    cursor.close()

    return df_rentals

In [65]:
# Suponiendo que 'conn' es tu conexión a MySQL establecida antes con mysql.connector

# Llamar a la función con esa conexión
df_rentals = rentals_month(conn, 5, 2005)  # Ejemplo: Recupera datos de alquiler para mayo de 2005

# Mostrar los resultados
df_rentals.head()

,rental_id,customer_id,staff_id,rental_date,return_date,inventory_id
0,1,130,1,2005-05-24 22:53:30,2005-05-26 22:04:30,367
1,2,459,1,2005-05-24 22:54:33,2005-05-28 19:40:33,1525
2,3,408,1,2005-05-24 23:03:39,2005-06-01 22:12:39,1711
3,4,333,2,2005-05-24 23:04:41,2005-06-03 01:43:41,2452
4,5,222,1,2005-05-24 23:05:21,2005-06-02 04:33:21,2079


In [66]:
#3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with 
    #the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year. 

In [67]:
def rental_count_month(df_rentals, month, year):
    """
    Calcula el número de alquileres realizados por cada customer_id durante el mes y año seleccionados.

    Parámetros:
    - df_rentals: DataFrame provisto por `rentals_month`.
    - month: El mes seleccionado.
    - year: El año seleccionado.

    Retorna:
    - Un DataFrame con `customer_id` y el correspondiente número de alquileres (`rental_count`).
    """
    # Asumiendo que df_rentals ya está filtrado por el mes y año correctos cuando se llamó a 'rentals_month'.
    # Si este DataFrame contiene otras fechas, te mostraré cómo filtrarlo correctamente a continuación:
    # Convertir 'rental_date' a datetime si aún no lo es.
    df_rentals['rental_date'] = pd.to_datetime(df_rentals['rental_date'])
    
    # Filtrar por mes y año (opcional, basado en la asunción anterior)
    filtered_df = df_rentals[df_rentals['rental_date'].dt.month == month]
    filtered_df = filtered_df[filtered_df['rental_date'].dt.year == year]
    
    # Agrupar por 'customer_id' y contar
    rental_counts = filtered_df.groupby('customer_id').size().reset_index(name='rental_count')

    return rental_counts

In [68]:
# Ejemplo de llamada a la función
# Asumiendo que "df_rentals" es tu DataFrame de alquileres
# y deseas contar los alquileres para mayo de 2005
month = 5
year = 2005

rental_counts = rental_count_month(df_rentals, month, year)

# Imprimir el resultado
rental_counts

,customer_id,rental_count
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


In [69]:
#4. Create a Python function called `compare_rentals` that takes two DataFrames as input containing the number of rentals made 
    #by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, 
    #which is the difference between the number of rentals in the two months.


In [70]:
import pandas as pd

def compare_rentals(df1, df2):
    """
    Compara el número de alquileres realizados por cada cliente en dos períodos diferentes.
    
    Parámetros:
    - df1 (DataFrame): DataFrame que contiene los alquileres del primer período.
    - df2 (DataFrame): DataFrame que contiene los alquileres del segundo período.
    
    Retorna:
    - DataFrame combinado con una columna nueva 'difference'.
    """
    # Asegurar que 'customer_id' sea el índice para facilitar la combinación
    df1.set_index('customer_id', inplace=True)
    df2.set_index('customer_id', inplace=True)
    
    # Combinar los DataFrames basándose en 'customer_id'
    combined_df = df1.join(df2, lsuffix='_first_period', rsuffix='_second_period', how='outer')
    
    # Calcular la diferencia en el número de alquileres
    combined_df['difference'] = combined_df['customer_id'] - combined_df['customer_id']
    
    # Manejar valores NaN resultantes de la operación para clientes que no aparecen en ambos períodos
    combined_df.fillna({'rental_count_first_period': 0, 'rental_count_second_period': 0, 'difference': 0}, inplace=True)
    
    return combined_df.reset_index()